## 0. 📦 Установка зависимостей

In [ ]:
!pip install numpy pandas torch tqdm
!pip install llama-index llama-index-embeddings-huggingface
!pip install llama-index-llms-openai-like llama-index-vector-stores-faiss
!pip install faiss-cpu nest-asyncio httpx openai
!pip install vllm

## 0.1 🚀 Запуск vLLM сервера

Перед запуском notebook необходимо запустить vLLM сервер в отдельном терминале


**Команда для запуска**
```bash
vllm serve Qwen/Qwen2.5-7B-Instruct \
    --host 0.0.0.0 \
    --port 8000 \
    --dtype auto \
    --gpu-memory-utilization 0.9
```

**Параметры:**
- `--host 0.0.0.0` — слушать на всех интерфейсах
- `--port 8000` — порт сервера (по умолчанию 8000)
- `--dtype auto` — автоматический выбор типа данных (float16/bfloat16)
- `--gpu-memory-utilization 0.9` — использование 90% GPU памяти


Сервер будет доступен по адресу `http://localhost:8000/v1`

## 1. 🔧 Конфигурация и Импорты

In [ ]:
import os

QUESTIONS_PATH = "data/questions.csv"
WEBSITES_PATH = "data/websites.csv"
FAISS_INDEX_PATH = "data/faiss_index"

# vLLM Server
VLLM_BASE_URL = "http://localhost:8000/v1"
LLM_MODEL = "Qwen/Qwen2.5-7B-Instruct"

TOP_K = 7
EMBEDDING_MODEL = "intfloat/multilingual-e5-large"

BREAKPOINT_PERCENTILE = 85
BUFFER_SIZE = 1

MAX_TOKENS = 200
TEMPERATURE = 0.1
BATCH_SIZE = 50

In [ ]:
import sys
import re
import numpy as np
import pandas as pd
import torch
from typing import List, Dict, Any, Optional
from tqdm.auto import tqdm

from llama_index.core import Document, VectorStoreIndex, Settings, StorageContext
from llama_index.core.node_parser import SemanticSplitterNodeParser, SentenceSplitter
from llama_index.core.schema import TextNode
from llama_index.core.prompts import PromptTemplate
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers import get_response_synthesizer

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai_like import OpenAILike
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

## 2. 📊 Загрузка и Предобработка Данных

In [ ]:
def load_data(questions_path: str, websites_path: str) -> tuple:
    """Загрузка данных с автоопределением разделителя"""
    try:
        df_websites = pd.read_csv(websites_path)
        if len(df_websites.columns) == 1:
            df_websites = pd.read_csv(websites_path, sep=';')
    except Exception as e:
        raise ValueError(f"Ошибка чтения websites: {e}")
    
    df_questions = pd.read_csv(questions_path)
    
    df_websites.columns = df_websites.columns.str.strip()
    df_questions.columns = df_questions.columns.str.strip()
    
    return df_websites, df_questions

df_websites, df_questions = load_data(QUESTIONS_PATH, WEBSITES_PATH)

print(f"Websites: {len(df_websites)} документов")
print(f"Questions: {len(df_questions)} вопросов")

In [ ]:
print("Пример документа (websites):")
print(df_websites.iloc[0])
print("\n" + "="*50)
print("\nПервые 5 вопросов:")
print(df_questions.head())

In [ ]:
def preprocess_text(text: str) -> str:
    """Базовая очистка текста"""
    if pd.isna(text):
        return ""
    text = str(text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

text_col = None
for col in ['text', 'web', 'content', 'body', 'page_content']:
    if col in df_websites.columns:
        text_col = col
        break

if text_col is None:
    raise KeyError(f"Не найдена колонка с текстом! Доступные: {list(df_websites.columns)}")

df_websites['clean_text'] = df_websites[text_col].apply(preprocess_text)

text_lengths = df_websites['clean_text'].str.len()
print(f"Статистика длины текстов: Min={text_lengths.min()}, Max={text_lengths.max()}, Mean={text_lengths.mean():.0f}")

## 3. 🧠 Semantic Chunking с LlamaIndex

In [ ]:
print(f"Загрузка embedding модели: {EMBEDDING_MODEL}...")

embed_model = HuggingFaceEmbedding(
    model_name=EMBEDDING_MODEL,
    device=device,
    embed_batch_size=32,
    trust_remote_code=True,
)

Settings.embed_model = embed_model

test_embedding = embed_model.get_text_embedding("Тестовая строка для проверки")
print(f"Embedding модель загружена. Размерность: {len(test_embedding)}")

In [ ]:
def create_documents(df: pd.DataFrame) -> List[Document]:
    """Создание LlamaIndex документов из DataFrame"""
    documents = []
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Creating documents"):
        text = row['clean_text']
        
        if len(text) < 50:
            continue
        
        metadata = {}
        if 'web_id' in row:
            metadata['web_id'] = str(row['web_id'])
        if 'url' in row:
            metadata['url'] = str(row['url'])
        if 'title' in row:
            metadata['title'] = str(row['title'])
        
        doc = Document(
            text=text,
            metadata=metadata,
            id_=str(row.get('web_id', idx))
        )
        documents.append(doc)
    
    return documents

documents = create_documents(df_websites)
print(f"Создано {len(documents)} документов из {len(df_websites)} записей")

In [ ]:
print(f"Запуск Semantic Chunking (breakpoint={BREAKPOINT_PERCENTILE}, buffer={BUFFER_SIZE})...")

semantic_splitter = SemanticSplitterNodeParser(
    embed_model=embed_model,
    breakpoint_percentile_threshold=BREAKPOINT_PERCENTILE,
    buffer_size=BUFFER_SIZE,
)

sentence_splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=100,
)

In [ ]:
all_nodes = []
failed_docs = []
CHUNK_BATCH_SIZE = 100

print(f"Обработка {len(documents)} документов...")

for i in tqdm(range(0, len(documents), CHUNK_BATCH_SIZE), desc="Chunking batches"):
    batch = documents[i:i + CHUNK_BATCH_SIZE]
    
    for doc in batch:
        try:
            if len(doc.text) > 200:
                nodes = semantic_splitter.get_nodes_from_documents([doc])
            else:
                nodes = [TextNode(text=doc.text, metadata=doc.metadata)]
            
            if not nodes:
                nodes = sentence_splitter.get_nodes_from_documents([doc])
            
            all_nodes.extend(nodes)
            
        except Exception as e:
            try:
                nodes = sentence_splitter.get_nodes_from_documents([doc])
                all_nodes.extend(nodes)
            except Exception as e2:
                failed_docs.append((doc.id_, str(e2)))

print(f"Создано {len(all_nodes)} чанков, ошибок: {len(failed_docs)}")

In [ ]:
chunk_lengths = [len(node.text) for node in all_nodes]

print(f"Статистика чанков: Всего={len(all_nodes)}, Min={min(chunk_lengths)}, Max={max(chunk_lengths)}, Mean={np.mean(chunk_lengths):.0f}")
print(f"\nПример чанка:\n{all_nodes[0].text[:500]}")

## 4. 🗂️ Построение FAISS Индекса

In [ ]:
print("Построение FAISS индекса...")

d = len(embed_model.get_text_embedding("test"))
print(f"Размерность: {d}")

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)

In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print(f"Индексация {len(all_nodes)} чанков...")

index = VectorStoreIndex(
    nodes=all_nodes,
    storage_context=storage_context,
    embed_model=embed_model,
    show_progress=True,
)

print(f"Индекс построен! Всего векторов: {faiss_index.ntotal}")

In [ ]:
os.makedirs(FAISS_INDEX_PATH, exist_ok=True)
index.storage_context.persist(persist_dir=FAISS_INDEX_PATH)
print(f"Индекс сохранён в {FAISS_INDEX_PATH}")

In [ ]:
retriever = index.as_retriever(similarity_top_k=TOP_K)

test_query = "Как узнать БИК и расчётный счёт?"
retrieved_nodes = retriever.retrieve(test_query)

print(f"Тестовый запрос: '{test_query}'")
print(f"Найдено {len(retrieved_nodes)} релевантных чанков:")

for i, node in enumerate(retrieved_nodes, 1):
    print(f"\n--- Чанк {i} (score: {node.score:.4f}) ---")
    print(node.text[:300] + "..." if len(node.text) > 300 else node.text)

## 5. 🤖 Настройка LLM (vLLM)

In [ ]:
OUTPUT_FOLDER = "llama_index_vllm"
OUTPUT_PATH = f"data/{OUTPUT_FOLDER}/submit.csv"
FULL_OUTPUT_PATH = f"data/{OUTPUT_FOLDER}/qa_full_dataset.csv"

In [ ]:
import requests

try:
    response = requests.get(f"{VLLM_BASE_URL}/models", timeout=5)
    if response.status_code == 200:
        models = response.json()
        print(f"vLLM сервер доступен. Модели: {[m['id'] for m in models['data']]}")
    else:
        print(f"vLLM сервер вернул статус {response.status_code}")
except requests.exceptions.ConnectionError:
    print(f"Ошибка: vLLM сервер недоступен по адресу {VLLM_BASE_URL}")
    print("Запустите сервер командой: vllm serve Qwen/Qwen2.5-7B-Instruct")

In [ ]:
print(f"Настройка LLM: {LLM_MODEL} через vLLM...")

llm = OpenAILike(
    model=LLM_MODEL,
    api_key="EMPTY",
    api_base=VLLM_BASE_URL,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    is_chat_model=True,
)

Settings.llm = llm

test_response = llm.complete("Скажи 'работает' если ты меня слышишь.")
print(f"LLM ответ: {test_response.text[:100]}")

## 6. 🔗 RAG Pipeline

In [ ]:
RAG_PROMPT = """Ты — помощник банка Альфа-Банк. Твоя задача — помочь клиенту, используя контекст.

КОНТЕКСТ:
{context_str}

ОБРАЩЕНИЕ КЛИЕНТА: {query_str}

ПРАВИЛА:
1. Если клиент описывает проблему или ситуацию (даже без вопроса) — дай совет как её решить.
2. Если вопрос о личных данных (баланс, операции, переводы клиента) — объясни где посмотреть: в мобильном приложении, личном кабинете Альфа-Онлайн, или позвонив на горячую линию 8 800 200-00-00.
3. Используй информацию из контекста для ответа.
4. Отвечай кратко: 1-3 предложения.
5. НЕ пиши "информации недостаточно" — лучше дай общий совет по теме.

ОТВЕТ:"""

qa_prompt = PromptTemplate(RAG_PROMPT)

In [ ]:
response_synthesizer = get_response_synthesizer(
    llm=llm,
    text_qa_template=qa_prompt,
    response_mode="compact",
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

print("Query Engine создан")

In [ ]:
test_queries = [
    "Как узнать БИК и расчётный счёт?",
    "Не приходит смс",
    "Что такое кэшбэк?",
    "Как войти в Альфа-Онлайн?",
]

print("Тестирование RAG Pipeline:\n")

for query in test_queries:
    response = query_engine.query(query)
    print(f"❓ {query}")
    print(f"💬 {response.response}")
    print("-" * 50)

## 7. 🚀 Генерация Ответов на Все Вопросы

In [ ]:
q_id_col = 'q_id' if 'q_id' in df_questions.columns else df_questions.columns[0]
query_col = 'query' if 'query' in df_questions.columns else df_questions.columns[1]

ids = df_questions[q_id_col].tolist()
queries = df_questions[query_col].tolist()

print(f"Подготовлено {len(queries)} вопросов")

In [ ]:
import asyncio
import nest_asyncio
import time
from asyncio import Semaphore
import httpx

nest_asyncio.apply()

BATCH_SIZE = 10
MAX_CONCURRENT = 10
RETRY_ATTEMPTS = 3
RETRY_DELAY = 2
TIMEOUT = 60


async def process_single_query(q_id, query, query_engine, retry=RETRY_ATTEMPTS):
    """Обработка одного запроса с retry"""
    for attempt in range(retry):
        try:
            response = await asyncio.wait_for(
                query_engine.aquery(query),
                timeout=TIMEOUT
            )
            answer = response.response.strip()
            context = "\n\n".join([node.text for node in response.source_nodes])
            
            return {
                "id": q_id,
                "question": query,
                "context": context,
                "answer": answer,
                "error": None
            }
        except asyncio.TimeoutError:
            if attempt < retry - 1:
                await asyncio.sleep(RETRY_DELAY * (attempt + 1))
                continue
            return {
                "id": q_id,
                "question": query,
                "context": "TIMEOUT",
                "answer": "К сожалению, информации недостаточно.",
                "error": "Timeout"
            }
        except Exception as e:
            if attempt < retry - 1:
                await asyncio.sleep(RETRY_DELAY * (attempt + 1))
                continue
            return {
                "id": q_id,
                "question": query,
                "context": "ERROR",
                "answer": "К сожалению, информации недостаточно.",
                "error": str(e)[:100]
            }


async def process_batch(batch_ids, batch_queries, query_engine):
    """Обработка батча запросов"""
    tasks = [
        process_single_query(q_id, query, query_engine)
        for q_id, query in zip(batch_ids, batch_queries)
    ]
    return await asyncio.gather(*tasks)


async def process_all_batched(ids, queries, query_engine):
    """Обработка всех запросов батчами"""
    all_results = []
    total_batches = (len(ids) + BATCH_SIZE - 1) // BATCH_SIZE
    
    semaphore = Semaphore(MAX_CONCURRENT)
    
    async def process_batch_with_semaphore(batch_idx):
        async with semaphore:
            start_idx = batch_idx * BATCH_SIZE
            end_idx = min(start_idx + BATCH_SIZE, len(ids))
            batch_ids = ids[start_idx:end_idx]
            batch_queries = queries[start_idx:end_idx]
            
            results = await process_batch(batch_ids, batch_queries, query_engine)
            return list(results)
    
    tasks = [process_batch_with_semaphore(i) for i in range(total_batches)]
    
    for i, coro in enumerate(tqdm(asyncio.as_completed(tasks), total=total_batches, desc="Processing batches")):
        batch_results = await coro
        all_results.extend(batch_results)
        
        if (i + 1) % 10 == 0:
            await asyncio.sleep(0.5)
    
    return all_results


print(f"Батчевая генерация ответов для {len(queries)} вопросов...")
print(f"Batch size: {BATCH_SIZE}, Max concurrent: {MAX_CONCURRENT}, Timeout: {TIMEOUT}s")

start_time = time.time()
all_results = asyncio.run(process_all_batched(ids, queries, query_engine))
elapsed = time.time() - start_time

print(f"Время выполнения: {elapsed:.1f} сек ({elapsed/60:.1f} мин)")
print(f"Скорость: {len(queries)/elapsed:.1f} запросов/сек")

all_results.sort(key=lambda x: x["id"])

results = [{"q_id": r["id"], "answer": r["answer"]} for r in all_results]
full_data = [{"q_id": r["id"], "question": r["question"], "context": r["context"], "answer": r["answer"]} for r in all_results]

errors = [r for r in all_results if r["error"]]
if errors:
    print(f"Ошибок: {len(errors)}")

print(f"Обработано {len(results)} вопросов")

## 8. 💾 Сохранение Результатов

In [ ]:
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

df_submit = pd.DataFrame(results)
df_submit.to_csv(OUTPUT_PATH, index=False)

print(f"Submission сохранён: {OUTPUT_PATH}")
print(f"Записей: {len(df_submit)}")
print(df_submit.head())

In [ ]:
df_full = pd.DataFrame(full_data)
df_full.to_csv(FULL_OUTPUT_PATH, index=False)

print(f"Полный датасет сохранён: {FULL_OUTPUT_PATH}")
print(f"Записей: {len(df_full)}, Колонки: {list(df_full.columns)}")

In [ ]:
insufficient_count = df_submit['answer'].str.contains(
    'недостаточно|Недостаточно', 
    case=False, 
    na=False
).sum()

answer_lengths = df_submit['answer'].str.len()

print(f"Статистика ответов:")
print(f"  Всего: {len(df_submit)}")
print(f"  'Информации недостаточно': {insufficient_count} ({insufficient_count/len(df_submit)*100:.1f}%)")
print(f"  Длина: Min={answer_lengths.min()}, Max={answer_lengths.max()}, Mean={answer_lengths.mean():.0f}")

In [ ]:
print("Случайные примеры ответов для проверки:\n")

sample = df_full.sample(min(10, len(df_full)))

for _, row in sample.iterrows():
    print(f"❓ {row['question']}")
    print(f"💬 {row['answer']}")
    print("-" * 50)

---

## 🎉 Готово!

Файлы сохранены:
- `data/llama_index_vllm/submit.csv` — submission file
- `data/llama_index_vllm/qa_full_dataset.csv` — полный датасет с контекстом
- `data/faiss_index/` — сохранённый индекс